In [1]:
#necessary imports
from termcolor import colored

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Dense


# Compare the two inputs
def comparator(learner, instructor):
    if len(learner) != len(instructor):
        raise AssertionError(f'Models does not have the same number of layers {len(learner)} != {len(instructor)}')
    for a, b in zip(learner, instructor):
        if tuple(a) != tuple(b):
            print(colored("Test failed", attrs=['bold']),
                  "\n Expected value \n\n", colored(f"{b}", "green"), 
                  "\n\n does not match the input value: \n\n", 
                  colored(f"{a}", "red"))
            raise AssertionError("Error in test") 
    print('\033[92mAll tests passed!')

# extracts the description of a given model
def summary(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    result = []
    for layer in model.layers:
        descriptors = [layer.__class__.__name__, layer.output_shape, layer.count_params()]
        if (type(layer) == Conv2D):
            descriptors.append(layer.padding)
            descriptors.append(layer.activation.__name__)
            descriptors.append(layer.kernel_initializer.__class__.__name__)
        if (type(layer) == MaxPooling2D):
            descriptors.append(layer.pool_size)
            descriptors.append(layer.strides)
            descriptors.append(layer.padding)
        if (type(layer) == Dropout):
            descriptors.append(layer.rate)
        if (type(layer) == ZeroPadding2D):
            descriptors.append(layer.padding)
        if (type(layer) == Dense):
            descriptors.append(layer.activation.__name__)
        result.append(descriptors)
    return result

In [31]:
import os
import numpy as np
import tensorflow as tf
import h5py
import math


def load_dataset():
    train_dataset = h5py.File('Sign_Dataset/train_signs.h5', "r")
    # your train set features
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(
        train_dataset["train_set_y"][:])  # your train set labels

    test_dataset = h5py.File('/Sign_Dataset/test_signs.h5', "r")
    # your test set features
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(
        test_dataset["test_set_y"][:])  # your test set labels

    classes = np.array(test_dataset["list_classes"][:])  # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size=64, seed=0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :, :, :]
    shuffled_Y = Y[permutation, :]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    # number of mini batches of size mini_batch_size in your partitionning
    num_complete_minibatches = math.floor(m / mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size: k *
                                  mini_batch_size + mini_batch_size, :, :, :]
        mini_batch_Y = shuffled_Y[k * mini_batch_size: k *
                                  mini_batch_size + mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches *
                                  mini_batch_size: m, :, :, :]
        mini_batch_Y = shuffled_Y[num_complete_minibatches *
                                  mini_batch_size: m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX

    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """

    # Retrieve the parameters from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    # Numpy Equivalents:
    # Z1 = np.dot(W1, X) + b1
    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    # Z2 = np.dot(W2, a1) + b2
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    # Z3 = np.dot(W3,Z2) + b3
    Z3 = tf.add(tf.matmul(W3, A2), b3)

    return Z3


def predict(X, parameters):

    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])

    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}

    x = tf.placeholder("float", [12288, 1])

    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)

    sess = tf.Session()
    prediction = sess.run(p, feed_dict={x: X})

    return prediction

In [3]:
from termcolor import colored
import tensorflow as tf
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
import numpy as np

def identity_block_test(target):
    np.random.seed(1)
    #X = np.random.randn(3, 4, 4, 6).astype(np.float32)
    X1 = np.ones((1, 4, 4, 3)) * -1
    X2 = np.ones((1, 4, 4, 3)) * 1
    X3 = np.ones((1, 4, 4, 3)) * 3

    X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

    A3 = target(X,
                f = 2,
                filters = [4, 4, 3],
                initializer=lambda seed=0:constant(value=1),
                training=False)


    A3np = A3.numpy()
    assert tuple(A3np.shape) == (3, 4, 4, 3), "Shapes does not match. This is really weird"
    assert np.all(A3np >= 0), "The ReLu activation at the last layer is missing"
    resume = A3np[:,(0,-1),:,:].mean(axis = 3)

    assert np.floor(resume[1, 0, 0]) == 2 * np.floor(resume[1, 0, 3]), "Check the padding and strides"
    assert np.floor(resume[1, 0, 3]) == np.floor(resume[1, 1, 0]),     "Check the padding and strides"
    assert np.floor(resume[1, 1, 0]) == 2 * np.floor(resume[1, 1, 3]), "Check the padding and strides"
    assert np.floor(resume[1, 1, 0]) == 2 * np.floor(resume[1, 1, 3]), "Check the padding and strides"

    assert resume[1, 1, 0] - np.floor(resume[1, 1, 0]) > 0.7, "Looks like the BatchNormalization units are not working"

    assert np.allclose(resume, 
                       np.array([[[0.0,       0.0,       0.0,        0.0],
                                  [0.0,       0.0,       0.0,        0.0]],
                                 [[192.71236, 192.71236, 192.71236,  96.85619],
                                  [ 96.85619,  96.85619,  96.85619,  48.9281 ]],
                                 [[578.1371,   578.1371,  578.1371,  290.56854],
                                  [290.56854,  290.56854, 290.56854, 146.78427]]]), atol = 1e-5 ), "Wrong values with training=False"
    
    np.random.seed(1)
    A4 = target(X,
                f = 3,
                filters = [3, 3, 3],
                initializer=lambda seed=7:constant(value=1),
                training=True)
    A4np = A4.numpy()
    resume = A4np[:,(0,-1),:,:].mean(axis = 3)
    assert np.allclose(resume, 
                         np.array([[[0.,         0.,        0.,      0.,        ],
                                  [0.,         0.,        0.,        0.,        ]],
                                 [[0.37394285, 0.37394285, 0.37394285, 0.37394285],
                                  [0.37394285, 0.37394285, 0.37394285, 0.37394285]],
                                 [[3.2379014,  4.1394243,  4.1394243,  3.2379014 ],
                                  [3.2379014,  4.1394243,  4.1394243,  3.2379014 ]]]), atol = 1e-5 ), "Wrong values with training=True"

    print('\033[92mAll tests passed!')

In [4]:
convolutional_block_output1 = [[[[0.,         0.66683817, 0.,         0.,        0.888539,   0.5274254 ],
   [0.,         0.65053666, 0.,         0.,         0.8959285,  0.49965227]],
  [[0.,         0.6312079,  0.,         0.,         0.86362475, 0.47643146],
   [0.,         0.56883204, 0.,        0.,         0.8553412,  0.417093 ]]],
 [[[1.0951002,  0.,         1.5761349,  0.6267836,  0.,         0.,        ],
   [1.3594234,  0.,         1.6074152,  0.6536726,  0.,         0.,        ]],
  [[0.49971345, 0.,         0.9446775,  0.6636579,  0.,         0.,        ],
   [0.98098886, 0.,         1.0731578,  0.6013661,  0.,         0.,        ]]],
 [[[3.2853007,  0.,         4.728405,   1.8803508,  0.,         0.,        ],
   [4.07827,    0.,         4.8222456,  1.9610177,  0.,         0.,        ]],
  [[1.4991404,  0.,         2.8340323,  1.9909737,  0.,         0.,        ],
   [2.9429667,  0.,         3.2194734,  1.8040984,  0.,         0.       ]]]]

convolutional_block_output2 = [[[[0.       , 2.7823157, 0.       , 0.       , 1.6960442, 2.8218517],
   [0.       , 1.5445004, 0.       , 0.       , 2.170656 , 1.3908148]],
  [[0.       , 1.9399526, 0.       , 0.       , 1.4798119, 1.9157798],
   [0.       , 0.       , 0.       , 0.9879823, 1.1234158, 0.       ]]],
 [[[0.,         0.4073585,  0.,         0.60906595, 0.61729676, 0.43824518],
   [0.,         0.4073585,  0.,         0.60906595, 0.61729676, 0.43824518]],
  [[0.,         0.4073585,  0.,         0.60906595, 0.61729676, 0.43824518],
   [0.,         0.4073585,  0.,         0.60906595, 0.61729676, 0.43824518]]],
 [[[2.6159182,  0.,         2.752994,   0.,         0.,         0.,        ],
   [3.4472604,  0.,         2.7312024,  0.03319526, 0.,         0.,        ]],
  [[0.9830525,  0.,         1.6564476,  0.9591365,  0.,         0.,        ],
   [2.5460882,  0.,         1.1291425,  0.,         0.,         0.,        ]]]]


ResNet50_summary =[['InputLayer', [(None, 64, 64, 3)], 0],
['ZeroPadding2D', (None, 70, 70, 3), 0, ((3, 3), (3, 3))],
['Conv2D', (None, 32, 32, 64), 9472, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 32, 32, 64), 256],
['Activation', (None, 32, 32, 64), 0],
['MaxPooling2D', (None, 15, 15, 64), 0, (3, 3), (2, 2), 'valid'],
['Conv2D', (None, 15, 15, 64), 4160, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 64), 36928, 'same', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 256), 16640, 'valid', 'linear', 'GlorotUniform'],
['Conv2D', (None, 15, 15, 256), 16640, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 15, 15, 256), 1024],
['BatchNormalization', (None, 15, 15, 256), 1024],
['Add', (None, 15, 15, 256), 0],
['Activation', (None, 15, 15, 256), 0],
['Conv2D', (None, 15, 15, 64), 16448, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 64), 36928, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 256), 16640, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 256), 1024],
['Add', (None, 15, 15, 256), 0],
['Activation', (None, 15, 15, 256), 0],
['Conv2D', (None, 15, 15, 64), 16448, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 64), 36928, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 64), 256],
['Activation', (None, 15, 15, 64), 0],
['Conv2D', (None, 15, 15, 256), 16640, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 15, 15, 256), 1024],
['Add', (None, 15, 15, 256), 0],
['Activation', (None, 15, 15, 256), 0],
['Conv2D', (None, 8, 8, 128), 32896, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 128), 147584, 'same', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 512), 66048, 'valid', 'linear', 'GlorotUniform'],
['Conv2D', (None, 8, 8, 512), 131584, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 8, 8, 512), 2048],
['BatchNormalization', (None, 8, 8, 512), 2048],
['Add', (None, 8, 8, 512), 0],
['Activation', (None, 8, 8, 512), 0],
['Conv2D', (None, 8, 8, 128), 65664, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 128), 147584, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 512), 66048, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 512), 2048],
['Add', (None, 8, 8, 512), 0],
['Activation', (None, 8, 8, 512), 0],
['Conv2D', (None, 8, 8, 128), 65664, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 128), 147584, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 512), 66048, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 512), 2048],
['Add', (None, 8, 8, 512), 0],
['Activation', (None, 8, 8, 512), 0],
['Conv2D', (None, 8, 8, 128), 65664, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 128), 147584, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 128), 512],
['Activation', (None, 8, 8, 128), 0],
['Conv2D', (None, 8, 8, 512), 66048, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 8, 8, 512), 2048],
['Add', (None, 8, 8, 512), 0],
['Activation', (None, 8, 8, 512), 0],
['Conv2D', (None, 4, 4, 256), 131328, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'GlorotUniform'],
['Conv2D', (None, 4, 4, 1024), 525312, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 4, 4, 256), 262400, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 4, 4, 256), 262400, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 4, 4, 256), 262400, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 4, 4, 256), 262400, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 4, 4, 256), 262400, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 256), 590080, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 256), 1024],
['Activation', (None, 4, 4, 256), 0],
['Conv2D', (None, 4, 4, 1024), 263168, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 4, 4, 1024), 4096],
['Add', (None, 4, 4, 1024), 0],
['Activation', (None, 4, 4, 1024), 0],
['Conv2D', (None, 2, 2, 512), 524800, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 512), 2359808, 'same', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 2048), 1050624, 'valid', 'linear', 'GlorotUniform'],
['Conv2D', (None, 2, 2, 2048), 2099200, 'valid', 'linear', 'GlorotUniform'],
['BatchNormalization', (None, 2, 2, 2048), 8192],
['BatchNormalization', (None, 2, 2, 2048), 8192],
['Add', (None, 2, 2, 2048), 0],
['Activation', (None, 2, 2, 2048), 0],
['Conv2D', (None, 2, 2, 512), 1049088, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 512), 2359808, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 2048), 1050624, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 2048), 8192],
['Add', (None, 2, 2, 2048), 0],
['Activation', (None, 2, 2, 2048), 0],
['Conv2D', (None, 2, 2, 512), 1049088, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 512), 2359808, 'same', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 512), 2048],
['Activation', (None, 2, 2, 512), 0],
['Conv2D', (None, 2, 2, 2048), 1050624, 'valid', 'linear', 'RandomUniform'],
['BatchNormalization', (None, 2, 2, 2048), 8192],
['Add', (None, 2, 2, 2048), 0],
['Activation', (None, 2, 2, 2048), 0],
['AveragePooling2D', (None, 1, 1, 2048), 0],
['Flatten', (None, 2048), 0],
['Dense', (None, 6), 12294, 'softmax']]

In [22]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow


%matplotlib inline

In [6]:
# UNQ_C1
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    cache = []
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    ### END CODE HERE

    return X

In [7]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

identity_block_test(identity_block)

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71236 192.71236 192.71236  96.85619]
  [ 96.85619  96.85619  96.85619  48.9281 ]]

 [[578.1371  578.1371  578.1371  290.56854]
  [290.56854 290.56854 290.56854 146.78427]]]
96.85619

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
All tests passed!


In [8]:
# UNQ_C2
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [9]:
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')

tf.Tensor(
[[[0.         0.66683817 0.         0.         0.888539   0.5274254 ]
  [0.         0.65053666 0.         0.         0.8959285  0.49965227]]

 [[0.         0.6312079  0.         0.         0.86362475 0.47643146]
  [0.         0.56883204 0.         0.         0.8553412  0.417093  ]]], shape=(2, 2, 6), dtype=float32)
All tests passed!


In [10]:
# UNQ_C3
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2, 2))(X)
    
    ### END CODE HERE

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [11]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization_20 (BatchN  (None, 32, 32, 64)  256         ['conv2d_20[0][0]']              
 ormalization)                                                                                

In [12]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

comparator(summary(model), ResNet50_summary)

All tests passed!


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [15]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 [==============================] - 211s 6s/step - loss: 2.1995 - accuracy: 0.4778
Epoch 2/10
34/34 [==============================] - 188s 6s/step - loss: 0.6381 - accuracy: 0.7769
Epoch 3/10
34/34 [==============================] - 194s 6s/step - loss: 0.5199 - accuracy: 0.8389
Epoch 4/10
34/34 [==============================] - 191s 6s/step - loss: 0.8184 - accuracy: 0.7074
Epoch 5/10
34/34 [==============================] - 194s 6s/step - loss: 0.3569 - accuracy: 0.8704
Epoch 6/10
34/34 [==============================] - 191s 6s/step - loss: 0.2200 - accuracy: 0.9259
Epoch 7/10
34/34 [==============================] - 191s 6s/step - loss: 0.3882 - accuracy: 0.8991
Epoch 8/10
34/34 [==============================] - 191s 6s/step - loss: 0.4464 - accuracy: 0.8722
Epoch 9/10
34/34 [==============================] - 191s 6s/step - loss: 0.1882 - accuracy: 0.9352
Epoch 10/10
34/34 [==============================] - 191s 6s/step - loss: 0.0702 - accuracy: 0.9741


In [16]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 3s 465ms/step - loss: 0.2114 - accuracy: 0.9417
Loss = 0.2113729566335678
Test Accuracy = 0.9416666626930237
